#### Sanity check 

In [1]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("docs/CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

/home/asetlur/anaconda3/envs/oclip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Label probs: tensor([[9.9950e-01, 4.1207e-04, 8.5318e-05]])


#### PCAM 

In [43]:
class AverageMeter:
    def __init__(self):
        self.num = 0
        self.val = 0

    def update(self, val, num):
        self.val += val * num
        self.num += num

    def get(self, percentage=False):
        val = self.val / self.num * 100 if percentage else self.val / self.num
        return val


In [12]:
import torchvision.datasets as datasets

# Specify the root directory where the dataset is located
root = "."

# Create an instance of the PCAM dataset
pcam_train_dataset = datasets.PCAM(root, download=True, split='train')
pcam_test_dataset = datasets.PCAM(root, download=True, split='test')


Downloading...
From (original): https://drive.google.com/uc?id=1qV65ZqZvWzuIVthK8eVDhIwrbnsJdbg_
From (redirected): https://drive.usercontent.google.com/download?id=1qV65ZqZvWzuIVthK8eVDhIwrbnsJdbg_&confirm=t&uuid=9e50135d-b2b9-4c4f-be64-80102e514b9a
To: /home/asetlur/open_clip_dp/pcam/camelyonpatch_level_2_split_test_x.h5.gz
100%|██████████| 801M/801M [00:07<00:00, 114MB/s]  
Downloading...
From: https://drive.google.com/uc?id=17BHrSrwWKjYsOgTMmoqrIjDy6Fa2o_gP
To: /home/asetlur/open_clip_dp/pcam/camelyonpatch_level_2_split_test_y.h5.gz
100%|██████████| 3.04k/3.04k [00:00<00:00, 3.03MB/s]


In [25]:
len(pcam_train_dataset), len(pcam_test_dataset)
pcam_train_dataset.transform = preprocess
pcam_test_dataset.transform = preprocess


In [32]:
# camelyon prompts

classes = [
    'lymph node',
    'lymph node containing metastatic tumor tissue',
]

templates = [
    'this is a photo of {classes[0]}',
    'this is a photo of {classes[1]}'
]

##### Test 0-shot

In [51]:
import torch
from PIL import Image
import open_clip
import tqdm

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
temperature = 100.
model = model.cuda()

test_loader = torch.utils.data.DataLoader(pcam_test_dataset, batch_size=32, shuffle=True)
text_features = model.encode_text(tokenizer(templates).cuda())
text_features /= text_features.norm(dim=-1, keepdim=True)
        
acc_meter = AverageMeter() 

progressbar = tqdm.tqdm(test_loader, total=len(test_loader))

for images, labels in progressbar:
    # Encode the image and text
    with torch.no_grad(), torch.cuda.amp.autocast():
        images = images.cuda()
        labels = labels.cuda()
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # Calculate the similarity scores
        text_probs = (temperature * image_features @ text_features.T).softmax(dim=-1)
        acc = text_probs.argmax(dim=-1) == labels
        acc_meter.update(acc.float().mean().item(), len(images))
        progressbar.set_description(f"Accuracy: {acc_meter.get(percentage=True):.4f}")
        # print(text_probs.shape, labels)
    

  0%|          | 0/1024 [00:00<?, ?it/s]


KeyboardInterrupt: 

##### Linear probe

In [ ]:
import torch
from PIL import Image
import open_clip
import tqdm

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
model = model.cuda()

train_loader = torch.utils.data.DataLoader(pcam_train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(pcam_test_dataset, batch_size=32, shuffle=True)

progressbar = tqdm.tqdm(train_loader, total=len(train_loader))
train_image_features = []
for images, _ in progressbar:
    # Encode the image and text
    with torch.no_grad(), torch.cuda.amp.autocast():
        images = images.cuda()
        image_features = model.encode_image(images)
        train_image_features.append(image_features.cpu())

train_image_features = torch.cat(train_image_features, dim=0)

import pickle
with open('train_features.pkl', 'wb') as f:
    pickle.dump(train_image_features, f)



progressbar = tqdm.tqdm(test_loader, total=len(test_loader))
test_image_features = []
for images, _ in progressbar:
    # Encode the image and text
    with torch.no_grad(), torch.cuda.amp.autocast():
        images = images.cuda()
        image_features = model.encode_image(images)
        test_image_features.append(image_features.cpu())


test_image_features = torch.cat(test_image_features, dim=0)


import pickle
with open('test_features.pkl', 'wb') as f:
    pickle.dump(test_image_features, f)